In [1]:
#Added Delta Normal VAR to the existing code for Momentum Investing

import numpy as np
import yfinance as yf
import pandas as pd

def get_sp500_tickers():
    table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    tickers = table[0]['Symbol'].tolist()
    return tickers

def fetch_and_analyze_data(ticker):
    try:
        adjusted_ticker = ticker.replace('.', '-')
        stock_data = yf.Ticker(adjusted_ticker).history(start='2023-12-29', end='2024-12-31')

        if not stock_data.empty:
            stock_data.index = stock_data.index.tz_localize(None)
            
            start_price = stock_data.iloc[0]['Close']
            end_price = stock_data.iloc[-1]['Close']
            annual_return = (end_price - start_price) / start_price * 100
            
            stock_data['Daily Return'] = stock_data['Close'].pct_change()
            std_dev = stock_data['Daily Return'].std() * (252**0.5)
            sharpe_ratio = (annual_return / 100 - risk_free_rate) / std_dev
            
            # Fetch P/E ratio using yfinance
            pe_ratio = yf.Ticker(adjusted_ticker).info.get('trailingPE', None)

            # Calculate Momentum Health Indicator
            stock_data['up_days'] = stock_data['Daily Return'] > 0
            stock_data['down_days'] = stock_data['Daily Return'] < 0

            up_days_percentage = stock_data['up_days'].sum() / len(stock_data) * 100
            down_days_percentage = stock_data['down_days'].sum() / len(stock_data) * 100

            stock_data['Momentum Health Indicator'] = up_days_percentage - down_days_percentage

            # Drop the temporary columns
            stock_data.drop(columns=['up_days', 'down_days'], inplace=True)
            
            # Calculate Delta-Normal VAR at 95% confidence level
            expected_return = stock_data['Daily Return'].mean()
            var_95 = expected_return - 1.96 * std_dev
            stock_data['VAR'] = var_95

            return {'Ticker': ticker, '2024 Return': annual_return, '2024 Std Dev': std_dev, 'Sharpe Ratio': sharpe_ratio, 'P/E Ratio': pe_ratio, 'Momentum Health Indicator': stock_data['Momentum Health Indicator'].iloc[-1], 'VAR': stock_data['VAR'].iloc[-1]}, stock_data
        else:
            return None, None
    except Exception as e:
        print(f"Skipping {ticker} due to error: {e}")
        return None, None

sp500_tickers = get_sp500_tickers()
returns_data = []
risk_free_rate = 0.02

writer = pd.ExcelWriter('sp500_data.xlsx', engine='openpyxl')

for ticker in sp500_tickers:
    result, stock_data = fetch_and_analyze_data(ticker)
    if result and stock_data is not None:
        returns_data.append(result)
        stock_data.to_excel(writer, sheet_name=result['Ticker'])

returns_df = pd.DataFrame(returns_data)
returns_df_sorted = returns_df.sort_values(by='2024 Return', ascending=False)
returns_df_sorted.to_excel(writer, sheet_name='Output', index=False)

writer.close()

print(returns_df_sorted[['Ticker', '2024 Return', '2024 Std Dev', 'Sharpe Ratio', 'P/E Ratio', 'Momentum Health Indicator', 'VAR']])


    Ticker  2024 Return  2024 Std Dev  Sharpe Ratio  P/E Ratio  \
361   PLTR   349.504950      0.643121      5.403413  336.30002   
476    VST   267.350480      0.567212      4.678151  31.458488   
348   NVDA   177.712900      0.525339      3.344752  53.719368   
461    UAL   138.027152      0.447542      3.039428  12.766388   
54    AXON   133.933356      0.449608      2.934409  146.90721   
..     ...          ...           ...           ...        ...   
178     EL   -48.145311      0.437380     -1.146492   131.1607   
93      CE   -55.216197      0.390918     -1.463637     6.4751   
250   INTC   -60.028963      0.512579     -1.210134       None   
320   MRNA   -60.402210      0.603295     -1.034356       None   
481    WBA   -61.633639      0.531370     -1.197540       None   

     Momentum Health Indicator       VAR  
361                   4.761905 -1.253743  
476                  18.650794 -1.105908  
348                  11.507937 -1.025047  
461                   3.174603 -0.8